## 1. 저장경로 설정

In [1]:
# 구글 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# konlpy 설치
!pip install konlpy

# 폰트 설치 

!apt -qq -y install fonts-nanum 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 40.1 MB/s eta 0:00:00
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 27 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 129502 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20180306-3_all.deb ...
Unpacking fonts-nanum (20180306-3) ...
Setting up fonts-nanum (20180306-3) ...
Processing triggers for fontconfig (2.13.1-2ubuntu3) ...


In [3]:
import pandas as pd 
import re
from konlpy.tag import Komoran
from collections import Counter
from wordcloud import WordCloud
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

file_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/'  # 파일패스

##2. 데이터 불러오기

In [4]:
import pickle

In [5]:
# 형태소 분석 된 파일 열기 (pickle로 저장)

with open(file_path + 'kiwi_pos.pkl', 'rb') as f:
    pos_lst = pickle.load(f)

In [6]:
morphs_lst = []
for sent in pos_lst:
    sent_lst = []
    for text, pos in sent:
        if pos == 'VV' or pos == 'VA':
            text = text + '다'
        if len(text) >= 2:
            sent_lst.append(text)
    morphs_lst.append(sent_lst)
morphs_lst

[['부랴부랴', '오다', '는데', '아무', '오다', '시간', '개념', '없다'],
 ['맞다', '사람', '진짜', '개념', '없다'],
 ['인방', '보다', '남자', '거르다', '맞다'],
 ['특히', '보다', '진짜', '거세', '시키', '어야'],
 ['댓글', '빠순', '몰려오다', '어서', '즈그', '주인', '쉴드', '치다', '보다'],
 ['이렇다', '어서', '인방', '인방', '하다'],
 ['에로', '배우', '데리다', '어다가', '아이돌', '만드다'],
 ['표정', '야동', 'ᆫ데'],
 ['솔직히', '여자', '48', 'kg', '이상', '지다', '어야', '아니'],
 ['그렇게', '따지다', '남자', '170', 'cm', '하도', '기다', '나오다', '되다'],
 ['중국', '달리다', '짐승'],
 ['짐승', '입장', '어야지', 'ㅋㅋ'],
 ['깐부', '라고', '부르다', '까지', '하다', '다니'],
 ['어머니', '이식', '싫다', '어서', '도망치다', '다고'],
 ['하다', '없다', '부모', '한테', '이식', '하다', '어도', '되다'],
 ['드디어', '하다'],
 ['빠지다', '힘들다', '어서', '같다'],
 ['보이다'],
 ['저것', '좌파', '정당', '대표', 'ᆫ지', '우파', '정당', '대표', 'ᆫ지', '분간', '가다'],
 ['요새', '온갖', '방법', '으로', '들어오다', '어서', '수준', '천차만별'],
 ['의사', '좋다', '많이', '벌다', '어서'],
 ['근데', '병원', '있다', '으면', '너무', '우울'],
 ['동안', '하다', '라고', '세금', '퍼주기', '거리', '두다', '밖에', '없다'],
 ['지금껏', '포퓰리즘', '정책', '밖에', '없다'],
 ['그렇다', '어도', '좋다', '사람', '구나'],
 ['비록',
 

In [7]:
# 형태소 분석 된 파일 열기 (pickle로 저장)

with open(file_path + 'kiwi_dataframe.pkl', 'rb') as f:
    kiwi_df = pickle.load(f)

kiwi_df

,분류,문장,형태소,점수,MORPHS,NOUNS,VERBnADJ
0,혐오,부랴부랴 왔는데 아무도 안왔네. 시간개념들이 없네,"[(부랴부랴, MAG), (오다, VV), (었, EP), (는데, EC), (아무...",10,"[부랴부랴, 오다, 었, 는데, 아무, 도, 안, 오다, 었, 네, 시간, 개념, ...","[아무, 시간, 개념]","[오다, 오다, 없다]"
1,혐오,맞아. 사람들이 진짜 개념이없다,"[(맞다, VV), (어, EF), (사람, NNG), (들, XSN), (이, J...",10,"[맞다, 어, 사람, 들, 이, 진짜, 개념, 이, 없다, 다]","[사람, 진짜, 개념]","[맞다, 없다]"
2,혐오,인방 보는 남자는 거르는게 맞다,"[(인방, NNG), (보다, VV), (는, ETM), (남자, NNG), (는,...",5,"[인방, 보다, 는, 남자, 는, 거르다, 는, 것, 이, 맞다, 다]","[인방, 남자, 것]","[보다, 거르다, 맞다]"
3,선정,특히 벗방보는 애들은 진짜 거세 시켜야함,"[(특히, MAG), (벗, NNG), (방, NNG), (보다, VV), (는, ...",10,"[특히, 벗, 방, 보다, 는, 애, 들, 은, 진짜, 거세, 시키, 어야, 하다, ᆷ]","[벗, 방, 애, 거세]","[보다, 하다]"
4,혐오,댓글에 빠순이들 몰려와서 즈그 주인님 쉴드치는 꼴 좀 봐,"[(댓글, NNG), (에, JKB), (빠순, NNG), (이, JKS), (들,...",10,"[댓글, 에, 빠순, 이, 들, 몰려오다, 어서, 즈그, 주인, 님, 쉴드, 치다,...","[댓글, 빠순, 즈그, 주인, 쉴드, 꼴]","[몰려오다, 치다, 보다]"
...,...,...,...,...,...,...,...
222240,혐오,그것보다 어떻게 밥그릇 챙기면서 살 수 있을지 고민하는 게 훨씬 이득이지 않아?,"[(그것, NP), (보다, JKB), (어떻다, VA), (게, EC), (밥그릇...",8,"[그것, 보다, 어떻다, 게, 밥그릇, 챙기다, 면서, 살다, ᆯ, 수, 있다, 을...","[그것, 밥그릇, 수, 고민, 것, 이득]","[어떻다, 챙기다, 살다, 있다, 이다, 않다]"
222241,혐오,저 나이 쳐먹고는 아이돌 하겠다고 나대는거임? 세상이 잘 돌아간다,"[(저, MM), (나이, NNG), (치다, VV), (어, EC), (먹다, V...",8,"[저, 나이, 치다, 어, 먹다, 고, 는, 아이돌, 하다, 겠, 다고, 나대다, ...","[나이, 아이돌, 거, 세상]","[치다, 먹다, 하다, 나대다, 이다, 돌아가다]"
222242,혐오,저런 애들을 뽑아주기라도 하겠음? 그냥 실패의 맛 보러 나온거 아님? ㅋ,"[(저런, MM), (애, NNG), (들, XSN), (을, JKO), (뽑다, ...",8,"[저런, 애, 들, 을, 뽑다, 어, 주다, 기, 이다, 라도, 하다, 겠, 음, ...","[애, 실패, 맛, 거]","[뽑다, 주다, 이다, 하다, 보다, 나오다, 아니다]"
222243,비악플,저 여자는 오늘도 레깅스 입네,"[(저, MM), (여자, NNG), (는, JX), (오늘, NNG), (도, J...",0,"[저, 여자, 는, 오늘, 도, 레깅스, 입다, 네]","[여자, 오늘, 레깅스]",[입다]


In [8]:
kiwi_df['MORPHS'] = morphs_lst

In [9]:
morphs_lst

[['부랴부랴', '오다', '는데', '아무', '오다', '시간', '개념', '없다'],
 ['맞다', '사람', '진짜', '개념', '없다'],
 ['인방', '보다', '남자', '거르다', '맞다'],
 ['특히', '보다', '진짜', '거세', '시키', '어야'],
 ['댓글', '빠순', '몰려오다', '어서', '즈그', '주인', '쉴드', '치다', '보다'],
 ['이렇다', '어서', '인방', '인방', '하다'],
 ['에로', '배우', '데리다', '어다가', '아이돌', '만드다'],
 ['표정', '야동', 'ᆫ데'],
 ['솔직히', '여자', '48', 'kg', '이상', '지다', '어야', '아니'],
 ['그렇게', '따지다', '남자', '170', 'cm', '하도', '기다', '나오다', '되다'],
 ['중국', '달리다', '짐승'],
 ['짐승', '입장', '어야지', 'ㅋㅋ'],
 ['깐부', '라고', '부르다', '까지', '하다', '다니'],
 ['어머니', '이식', '싫다', '어서', '도망치다', '다고'],
 ['하다', '없다', '부모', '한테', '이식', '하다', '어도', '되다'],
 ['드디어', '하다'],
 ['빠지다', '힘들다', '어서', '같다'],
 ['보이다'],
 ['저것', '좌파', '정당', '대표', 'ᆫ지', '우파', '정당', '대표', 'ᆫ지', '분간', '가다'],
 ['요새', '온갖', '방법', '으로', '들어오다', '어서', '수준', '천차만별'],
 ['의사', '좋다', '많이', '벌다', '어서'],
 ['근데', '병원', '있다', '으면', '너무', '우울'],
 ['동안', '하다', '라고', '세금', '퍼주기', '거리', '두다', '밖에', '없다'],
 ['지금껏', '포퓰리즘', '정책', '밖에', '없다'],
 ['그렇다', '어도', '좋다', '사람', '구나'],
 ['비록',
 

In [10]:
type(morphs_lst)

list

In [10]:
len(morphs_lst)

222245

## 3. GLOVE 학습

### 1) GloVe 모델 불러오기

In [11]:
pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.9/974.9 KB 52.4 MB/s eta 0:00:00


In [12]:
# import 해서 불러오기
from glove import Corpus, Glove
import numpy as np
import csv

### 2) Corpus 만들기

- Corpus : vocab(corpus, 단어사전) 을 만들어주는 기능

In [13]:
# corpus 생성
corpus = Corpus()
corpus.fit(morphs_lst, window=20)

### 3) GloVe 학습

In [14]:
# model
glove = Glove(no_components=128, learning_rate=0.01)     # 128차원으로 임베딩된 벡터로 나온도록 설정
%time glove.fit(corpus.matrix, epochs=50, no_threads=4, verbose=False)   # 위에서 만들어준 corpus를 matrix 형태로 넣어줌  # Wall time: 2min 36s
glove.add_dictionary(corpus.dictionary)

# save
glove.save(file_path +'/kiwi_glove_w20_epoch50.model')

CPU times: user 4min 38s, sys: 237 ms, total: 4min 38s
Wall time: 2min 30s


### 4) GloVe 모델 불러오고 word_dictionary 만들기

In [15]:
# load glove
glove_model = Glove.load(file_path + '/kiwi_glove_w20_epoch50.model')  #glvoe 모델이 단어들에 대해서 어떤 임베딩 벡터를 내보내는지 확인

In [17]:
# word dict 생성
word_dict = {}
for word in  glove_model.dictionary.keys():
    word_dict[word] = glove_model.word_vectors[glove_model.dictionary[word]]
print('[Success !] Lengh of word dict... : ', len(word_dict))

# save word_dict
with open(file_path + '/glove_word_dict.pickle', 'wb') as f:
    pickle.dump(word_dict, f)
print('[Success !] Save word dict!...')

[Success !] Lengh of word dict... :  34119
[Success !] Save word dict!...


### 5) 임베딩된 단어 벡터 확인하기

In [18]:
word_dict["되다"]

array([ 0.26354427,  0.21332482,  0.2292347 , -0.20153509, -0.10777738,
        0.20763145, -0.19114818, -0.2059278 ,  0.34920241, -0.22079755,
       -0.21252349, -0.18100646,  0.22163128,  0.21585306, -0.35932423,
        0.17908614, -0.19868207,  0.14752454,  0.23334376,  0.20346133,
        0.2261939 , -0.34383847, -0.19815417, -0.22235691,  0.21160212,
       -0.23460724, -0.21984538,  0.26760863,  0.14765072,  0.19036946,
       -0.13469055,  0.21382975, -0.14306329, -0.19244836,  0.25327844,
       -0.2201426 , -0.19572373,  0.13009777,  0.19595354,  0.20743967,
        0.21005791, -0.2373827 ,  0.20833689,  0.24304442, -0.18242554,
       -0.14115434, -0.20560374,  0.12970999,  0.22287259, -0.19143408,
       -0.19256225, -0.23608467, -0.21746147,  0.20077633, -0.1600344 ,
        0.16689034,  0.17056494, -0.22572558, -0.2353139 ,  0.24823502,
        0.17365249,  0.16439353,  0.21999509,  0.22595464,  0.1951379 ,
        0.24757124,  0.1714387 , -0.20028777,  0.26014118, -0.24

In [ ]:
# word dict : train데이터의 임베딩 사전
#total_word_dict = {}
#cnt = 0

#for word in test_tokens:
#    if word in word_dict.keys():
#        total_word_dict[word] = word_dict[word]
#    else:
#        word_dict[word] = np.zeros((128))
#        cnt += 1  # 처음 본 단어 갯수 세기

#print('no train word -> 0....', cnt)
#print('token -> word embedding....!',len(unique_tokens))

### 6) 워드 임베딩된 벡터로 문장 단위 임베딩

In [19]:
def sent2vec_glove(tokens, embedding_dim=128):
    '''문장 token 리스트를 받아서 임베딩 시킨다.'''
  
    size = len(tokens)
    matrix = np.zeros((size, embedding_dim))
    word_table = word_dict     # glove word_dict

    for i, token in enumerate(tokens):
        vector = np.array([
            word_table[t] for t in token
            if t in word_table
        ])

        if vector.size != 0:
            final_vector = np.mean(vector, axis=0)
            matrix[i] = final_vector

    return matrix

## 7. 유사한 단어 뽑기

- glove.most_similar() : 입력 단어의 가장 유사한 단어들의 리스트를 리턴

### 1) 하나씩 Test

In [ ]:
# glove.most_similar("김치련",number=10)

In [ ]:
# glove.most_similar("칼",number=10)

In [ ]:
#glove.most_similar("싯발",number=10)

In [19]:
# 소수점 이하 4자리만 남기기
def GloVeSimilar(text, n=10):
    sim1=glove.most_similar(text,number=10)
    sim1 = [(word, round(p,4)) for word, p in sim1]
    return sim1

In [ ]:
glove.most_similar("머저리",number=10) #혐오

[('ㅅㄲ', 0.9872937868166978),
 ('저능아', 0.9807596973631454),
 ('버러지', 0.980574099691137),
 ('녀석', 0.9779240342531236),
 ('대단', 0.9755373135181235),
 ('짐승', 0.9752079372076043),
 ('그지', 0.9751571035350478),
 ('멍청이', 0.9725751144379035),
 ('바보', 0.966346053216156)]

In [ ]:
glove.most_similar("그지",number=10) #혐오

[('녀석', 0.9762929817153552),
 ('머저리', 0.9751571035350479),
 ('버러지', 0.9672390303873937),
 ('존재', 0.9634334926734135),
 ('샛기', 0.9618742731764512),
 ('ㅅㄲ', 0.9598789082333751),
 ('개념', 0.9568038328162548),
 ('다니까', 0.955494195275537),
 ('라서', 0.9525241648335484)]

### 2) 리스트에 있는 단어 넣기

In [20]:
# 방법 : 키워드 리스트를 넣었을 때 상위 n개 가져오기
# input: keywordlist=[(keyword1, tag1), (keyword2, tag2), ...]
def ExtractSimilar(word_lst, n, model_name):
    stopwords = ['다']
    sim_lst = []
    sim_set = set()
    for key, tag in word_lst:
        sim = model_name.most_similar(key, number=n)
        for word, p in sim:
            if word[0].isdigit()==False and len(word) > 1 and word not in sim_set:
              if p > 0.8 and word[-1] not in stopwords:
                sim_lst.append([word, key, round(p,4), tag, len(word)])
                sim_set.add(word)

    sim_df = pd.DataFrame(sim_lst, columns=['keyword', 'search', 'similarity', 'tag', 'len'])
    sim_df.sort_values(by=['search','keyword','len','similarity'], inplace=True)
    sim_df.reset_index(drop=True, inplace=True)
    
    return sim_df

In [21]:
# 파일 열기
with open(file_path + 'word_noun_lst.pkl', 'rb') as f:
    sexual_noun_lst, violence_noun_lst, hate_noun_lst = pickle.load(f)

print(sexual_noun_lst)
print(violence_noun_lst)
print(hate_noun_lst)

['큼', '후궁아', '빠딱이', '주물', '처세술', '췡', '나위', '왕잣탱', '빼액거리지', '획득', '보털', '치셈', '임펙트', '치토스', '꽃휴', '쫨지', '국내선', '까만색', '솜털', '센세', '쌍방울', '취향저격', '핡', '유희장', '앗싸', '허덕거리게', '무조건', '페미여', '펫캠', '어케하누', '유희장', '보확찢', '뾰잉', '오럴', '모솔드립', '보쥬', '까이냐', '소추라서', '엣헴거려', '테크닉', '쎄빠', '쑤컹쑤컹', '한녀피싸개임', '쩔음', '로맨티스트', '승', '테크닉', '바이브레이터', '영', '탄', '영', '리라', '크네', '돌쇠', '씰룩대', '되세', '아작남', '대꼴', '오피녀임', '싸지르셈', '돌쇠', '응꼬', '체액', '청량리', '보짓꾸녕', '쭈물럭', '쭈물럭', '매직', '본드걸', '무브', '슴', '빤쮸', '버선', '재기하네', '강직', '섹파', '섹파', '뷰지련', '챙녀촌', '챙녀임', '췡련', '액윽액윽', '알카에다', '샹들리에', '피토', '섹파', '보픈업', '십창련', '푼거니', '러브젤', '의젖이다', '똥꼬털', '투서', '액받', '무녀', '간택', '패디큐어', '저쪼', '애원', '대줘', '밖아서', '치토스', '구인', '세로', '초이스', '도랏네', '즐', '치토스임', '치토스', '개꼴림', '시몬스', '앞부분', '꼬돌이에', '짧뚱', '주절먹', '확찢', '유일무이', '연년생', '세어가며서', '조수미', '상', '까라', '뭉모씨', '애첩', '검두', '엄지손가락', '길지', '룸망주', '거시기', '허벌년', '뱅기', '뒤태', '클로즈업', '무드', '별풍녀', '리코타', '품번', '노꼴', '노식', '딸따리', '클리', '응꼬', '보찢좌', '섹시녀', '머꼴임', '매매녀', '스벌년', '보토', '노꼴임',

## 선정

In [ ]:
sexual_list=[(noun, '선정') for noun in sexual_noun_lst if len(noun) > 1 and noun in glove_model.dictionary.keys()]
rep = 5
print('Start!', end=' ')

for i in range(rep):
    sexual_dic_df = ExtractSimilar(sexual_list, n=10, model_name=glove)
    print(f'> {sexual_dic_df.shape[0]}', end=' ')
    sexual_list=[(noun, '선정') for noun in sexual_dic_df['keyword'] if len(noun) > 1 and noun in glove_model.dictionary.keys()]
print('End!')

sexual_dic_df

Start! > 3694 > 3473 > 3225 > 3519 > 3873 End!


,keyword,search,similarity,tag,len
0,노숙,AV,0.9659,선정,2
1,노동자,BJ,0.8765,선정,3
2,다기,BJ,0.8735,선정,2
3,쌩얼,BJ,0.8758,선정,2
4,웬만,BJ,0.9232,선정,2
...,...,...,...,...,...
3868,딜러,히어로,0.9873,선정,2
3869,안구,히어로,0.9870,선정,2
3870,몸값,히틀러,0.9814,선정,2
3871,아프칸,히틀러,0.9809,선정,3


In [ ]:
cat_lst = ['비악플','선정','폭력','혐오']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in sexual_dic_df['keyword'].tolist():
    counts = kiwi_df.loc[kiwi_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])
    max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 100 == 0:
        print(f'> {i}', end=' ')
    if i % 1000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
sexual_dic_df = pd.concat([sexual_dic_df, cnt_df], axis=1)
sexual_dic_df

Start! > 100 > 200 > 300 > 400 > 500 > 600 > 700 > 800 > 900 > 1000 
> 1100 > 1200 > 1300 > 1400 > 1500 > 1600 > 1700 > 1800 > 1900 > 2000 
> 2100 > 2200 > 2300 > 2400 > 2500 > 2600 > 2700 > 2800 > 2900 > 3000 
> 3100 > 3200 > 3300 > 3400 > 3500 > 3600 > 3700 > 3800 > 3900 > 4000 
> 4100 > 4200 > 4300 > 4400 > 4500 > 4600 End!


,keyword,search,similarity,tag,len,빈도기반 tag,비악플,선정,폭력,혐오
0,노숙,AV,0.9659,선정,2,혐오,5,0,2,18
1,음란,AV,0.9684,선정,2,비악플,7,5,0,5
2,주부,AV,0.9625,선정,2,비악플,33,0,1,17
3,보톡스,AZ,0.8972,선정,3,비악플,12,0,0,6
4,입장료,AZ,0.8910,선정,3,비악플,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...
4683,가혹,히틀러,0.9813,선정,2,비악플,17,0,3,4
4684,딸래미,히틀러,0.9820,선정,3,혐오,4,3,1,6
4685,아프칸,히틀러,0.9809,선정,3,비악플,4,0,0,3
4686,폭주족,히틀러,0.9814,선정,3,비악플,8,0,7,8


In [ ]:
sexual_list=[(noun, '선정') for noun in sexual_noun_lst if len(noun) > 1 and noun in glove_model.dictionary.keys()]
rep = 7
print('Start!', end=' ')

for i in range(rep):
    sexual_dic_df = ExtractSimilar(sexual_list, n=10, model_name=glove)
    print(f'> {sexual_dic_df.shape[0]}', end=' ')
    sexual_list=[(noun, '선정') for noun in sexual_dic_df['keyword'] if len(noun) > 1 and noun in glove_model.dictionary.keys()]
print('End!')

sexual_dic_df

Start! > 3694 > 3473 > 3225 > 3519 > 3873 > 4298 > 4688 End!


,keyword,search,similarity,tag,len
0,노숙,AV,0.9659,선정,2
1,음란,AV,0.9684,선정,2
2,주부,AV,0.9625,선정,2
3,보톡스,AZ,0.8972,선정,3
4,입장료,AZ,0.8910,선정,3
...,...,...,...,...,...
4683,가혹,히틀러,0.9813,선정,2
4684,딸래미,히틀러,0.9820,선정,3
4685,아프칸,히틀러,0.9809,선정,3
4686,폭주족,히틀러,0.9814,선정,3


In [ ]:
sexual_dic_df[['keyword', '빈도기반 tag', '비악플', '선정', '폭력', '혐오']].to_csv(file_path + 'eunji_sexual_dic.txt', sep='\t', header=True, index=False)

In [ ]:
sorted(counts)

['비악플', '선정', '폭력', '혐오']

In [ ]:
key = '종특'
cat_lst = ['비악플','선정','혐오','폭력']
counts = kiwi_df.loc[kiwi_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts().reset_index()
if len(counts) != len(cat_lst):
  add_cat = [[cat, 0] for cat in cat_lst if cat not in counts['분류'].tolist()]
  for cat in add_cat:
    counts.loc[len(counts)] = cat

counts

,분류,0
0,혐오,11
1,비악플,1
2,폭력,1
3,선정,0


In [ ]:
# glove.most_similar('한남', 10)
print(glove_model.dictionary.keys())

dict_keys(['부랴부랴', '오다', '는데', '아무', '시간', '개념', '없다', '맞다', '사람', '진짜', '인방', '보다', '남자', '거르다', '특히', '거세', '시키', '어야', '댓글', '빠순', '몰려오다', '어서', '즈그', '주인', '쉴드', '치다', '이렇다', '하다', '에로', '배우', '데리다', '어다가', '아이돌', '만드다', '표정', '야동', 'ᆫ데', '솔직히', '여자', '48', 'kg', '이상', '지다', '아니', '그렇게', '따지다', '170', 'cm', '하도', '기다', '나오다', '되다', '중국', '달리다', '짐승', '입장', '어야지', 'ㅋㅋ', '깐부', '라고', '부르다', '까지', '다니', '어머니', '이식', '싫다', '도망치다', '다고', '부모', '한테', '어도', '드디어', '빠지다', '힘들다', '같다', '보이다', '저것', '좌파', '정당', '대표', 'ᆫ지', '우파', '분간', '가다', '요새', '온갖', '방법', '으로', '들어오다', '수준', '천차만별', '의사', '좋다', '많이', '벌다', '근데', '병원', '있다', '으면', '너무', '우울', '동안', '세금', '퍼주기', '거리', '두다', '밖에', '지금껏', '포퓰리즘', '정책', '그렇다', '구나', '비록', '가지다', '놀다', '지만', '미치다', '패다', '으니까', '당당', '살다', '읍니다', '이번', '트레이너', '회원', '랭킹', '찍다', '다며', '기생', '오래비', '같이', '빼다', '반반', '지배', '취향', '제격', 'ㅉㅉ', '음악', '성공', '알다', '그니까', 'ᆫ다고', '박히다', '등골', '빨다', '먹다', '재수', '못생기다', '입술', '아프리카인', '잖아', '맨날', '다이어트', '실패', '한심', '아줌마', '

In [ ]:
# 선정 사전 저장
sexual_dic_df.to_csv(file_path + 'eunji_sexual_dic.txt', header=False, index=False)

### 폭력

In [26]:
# 반복 함수
violence_list=[(noun, '폭력') for noun in violence_noun_lst if len(noun) > 1 and noun in glove_model.dictionary.keys()]

rep = 5
print('Start!', end=' ')
for i in range(rep):
    vio_dic_df = ExtractSimilar(violence_list, n=10, model_name=glove)
    print(f'> {vio_dic_df.shape[0]}', end=' ')

    violence_list=[(noun, '폭력') for noun in vio_dic_df['keyword'] if len(noun) > 1 and noun in glove_model.dictionary.keys()]
print('End!')
vio_dic_df

Start! > 5468 > 4036 > 3281 > 3254 > 3475 End!


,keyword,search,similarity,tag,len
0,뮤지컬,AV,0.9699,폭력,3
1,아동,AV,0.9631,폭력,2
2,음란,AV,0.9695,폭력,2
3,주부,AV,0.9581,폭력,2
4,꼬맹이,DNA,0.9910,폭력,3
...,...,...,...,...,...
3470,남경,히틀러,0.9843,폭력,2
3471,선조,히틀러,0.9817,폭력,2
3472,유교,히틀러,0.9806,폭력,2
3473,쭉쭉,히틀러,0.9806,폭력,2


In [27]:
# 새로운 태그 만들기
cat_lst = ['비악플','선정','폭력','혐오']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in vio_dic_df['keyword'].tolist():
    counts = kiwi_df.loc[kiwi_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])
    max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 100 == 0:
        print(f'> {i}', end=' ')
    if i % 1000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
vio_dic_df = pd.concat([vio_dic_df, cnt_df], axis=1)
vio_dic_df

Start! > 100 > 200 > 300 > 400 > 500 > 600 > 700 > 800 > 900 > 1000 
> 1100 > 1200 > 1300 > 1400 > 1500 > 1600 > 1700 > 1800 > 1900 > 2000 
> 2100 > 2200 > 2300 > 2400 > 2500 > 2600 > 2700 > 2800 > 2900 > 3000 
> 3100 > 3200 > 3300 > 3400 End!


,keyword,search,similarity,tag,len,빈도기반 tag,비악플,선정,폭력,혐오
0,뮤지컬,AV,0.9699,폭력,3,비악플,16,0,2,5
1,아동,AV,0.9631,폭력,2,비악플,49,2,13,33
2,음란,AV,0.9695,폭력,2,비악플,7,5,0,5
3,주부,AV,0.9581,폭력,2,비악플,33,0,1,17
4,꼬맹이,DNA,0.9910,폭력,3,비악플,7,2,2,6
...,...,...,...,...,...,...,...,...,...,...
3470,남경,히틀러,0.9843,폭력,2,비악플,11,2,0,6
3471,선조,히틀러,0.9817,폭력,2,혐오,2,0,3,10
3472,유교,히틀러,0.9806,폭력,2,혐오,8,0,2,26
3473,쭉쭉,히틀러,0.9806,폭력,2,선정,6,19,2,6


In [28]:
# 저장하기 
vio_dic_df[['keyword', '빈도기반 tag', '비악플', '선정', '폭력', '혐오']].to_csv(file_path + 'eunji_violence_dic.txt', sep='\t', header=True, index=False)

### 혐오

In [22]:
# 반복 함수
hate_list=[(noun, '혐오') for noun in hate_noun_lst if len(noun) > 1 and noun in glove_model.dictionary.keys()]
rep = 5
print('Start!', end=' ')
for i in range(rep):
    hate_dic_df = ExtractSimilar(hate_list, n=10, model_name=glove)
    print(f'> {hate_dic_df.shape[0]}', end=' ')

    hate_list=[(noun, '혐오') for noun in hate_dic_df['keyword'] if len(noun) > 1 and noun in glove_model.dictionary.keys()]
print('End!')
hate_dic_df

Start! > 9172 > 5576 > 4474 > 4516 > 4816 End!


,keyword,search,similarity,tag,len
0,노숙,AV,0.9657,혐오,2
1,공중파,BJ,0.8716,혐오,3
2,폐급,DNA,0.9913,혐오,2
3,알페스,DP,0.9701,혐오,3
4,머리채,GDP,0.9907,혐오,3
...,...,...,...,...,...
4811,치사,흡연실,0.9882,혐오,2
4812,업종,흡연자,0.9698,혐오,2
4813,불임,흡입,0.9887,혐오,2
4814,단골,희대,0.9869,혐오,2


In [24]:
# 새로운 태그 만들기
cat_lst = ['비악플','선정','폭력','혐오']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in hate_dic_df['keyword'].tolist():
    counts = kiwi_df.loc[kiwi_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])
    max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 100 == 0:
        print(f'> {i}', end=' ')
    if i % 1000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
hate_dic_df = pd.concat([hate_dic_df, cnt_df], axis=1)
hate_dic_df

Start! > 100 > 200 > 300 > 400 > 500 > 600 > 700 > 800 > 900 > 1000 
> 1100 > 1200 > 1300 > 1400 > 1500 > 1600 > 1700 > 1800 > 1900 > 2000 
> 2100 > 2200 > 2300 > 2400 > 2500 > 2600 > 2700 > 2800 > 2900 > 3000 
> 3100 > 3200 > 3300 > 3400 > 3500 > 3600 > 3700 > 3800 > 3900 > 4000 
> 4100 > 4200 > 4300 > 4400 > 4500 > 4600 > 4700 > 4800 End!


,keyword,search,similarity,tag,len,빈도기반 tag,비악플,선정,폭력,혐오
0,노숙,AV,0.9657,혐오,2,혐오,5,0,2,18
1,공중파,BJ,0.8716,혐오,3,비악플,15,1,1,13
2,폐급,DNA,0.9913,혐오,2,혐오,0,0,4,8
3,알페스,DP,0.9701,혐오,3,혐오,5,1,2,7
4,머리채,GDP,0.9907,혐오,3,폭력,1,2,12,2
...,...,...,...,...,...,...,...,...,...,...
4811,치사,흡연실,0.9882,혐오,2,혐오,9,2,0,15
4812,업종,흡연자,0.9698,혐오,2,비악플,11,0,0,3
4813,불임,흡입,0.9887,혐오,2,혐오,3,0,4,9
4814,단골,희대,0.9869,혐오,2,비악플,9,3,0,9


In [25]:
# 저장하기 
hate_dic_df[['keyword', '빈도기반 tag', '비악플', '선정', '폭력', '혐오']].to_csv(file_path + 'eunji_hate_dic.txt', sep='\t', header=True, index=False)

### 3) 사전 저장

### 4) 모델 저장

In [ ]:
# 모델저장
model.save_word2vec_format('Glove_model') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("Glove_model") # 모델 로드